<a href="https://colab.research.google.com/github/Sithucoder/Plagiarism_Detector/blob/main/Plagairism_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import nltk
nltk.download("popular")
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import string
from nltk.corpus import stopwords
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer


[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

**Load Data**

In [4]:
data = pd.read_csv("train_snli.csv")
data.head()

,Column1,Column2,Column3
0,A person on a horse jumps over a broken down a...,"A person is at a diner, ordering an omelette.",0
1,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse.",1
2,Children smiling and waving at camera,There are children present,1
3,Children smiling and waving at camera,The kids are frowning,0
4,A boy is jumping on skateboard in the middle o...,The boy skates down the sidewalk.,0


In [7]:
data['Column3'].value_counts()

,count
Column3,
0,183966
1,183407


In [14]:
def preprocess_text(text):
    # Check if text is a string
    if isinstance(text, str):
        # Remove punctuation
        text = text.translate(str.maketrans("", "", string.punctuation))
        # Convert to lowercase
        text = text.lower()
        # Remove stop words
        stop_words = set(stopwords.words("english"))
        text = " ".join(word for word in text.split() if word not in stop_words)
    # If text is not a string (e.g., float or NaN), return it as is
    # or handle it in a way that makes sense for your data
    # For example, you could convert it to an empty string:
    # else:
    #     text = ''
    return text

data["source_text"] = data["source_text"].apply(preprocess_text)
data["plagiarized_text"] = data["plagiarized_text"].apply(preprocess_text)

In [16]:
# Before applying TF-IDF, handle missing values (NaN) in 'Column1' and 'Column2'
# Replace with empty strings:
data["source_text"] = data["source_text"].fillna("")
data["plagiarized_text"] = data["plagiarized_text"].fillna("")

# Now proceed with TF-IDF vectorization:
tfidf_vectorizer = TfidfVectorizer()
X = tfidf_vectorizer.fit_transform(data["source_text"] + " " + data["plagiarized_text"])

In [17]:
y = data["label"]

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# **Logistic Regression**

In [19]:
# Import necessary libraries
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix



# Now you can train your model
model = LogisticRegression(C=0.8, penalty='l2', solver='saga', max_iter=600)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

cm = confusion_matrix(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_rep)
print("Confusion Matrix")
print(cm)


Accuracy: 0.8243243243243243
Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.86      0.82        35
           1       0.86      0.79      0.83        39

    accuracy                           0.82        74
   macro avg       0.83      0.83      0.82        74
weighted avg       0.83      0.82      0.82        74

Confusion Matrix
[[30  5]
 [ 8 31]]


# **Random Forest Model**

In [23]:
from sklearn.ensemble import RandomForestClassifier
# Instantiate the model
model = RandomForestClassifier(n_estimators=9000, n_jobs=-1, random_state=42)
# Fit the model
model.fit(X_train, y_train)
# Make predictions
y_pred = model.predict(X_test)
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
# Generate classification report
classification_rep = classification_report(y_test, y_pred)
# Generate confusion matrix
cm = confusion_matrix(y_test, y_pred)
# Print results
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_rep)
print("Confusion Matrix:")
print(cm)

Accuracy: 0.8108108108108109
Classification Report:
              precision    recall  f1-score   support

           0       0.72      0.97      0.83        35
           1       0.96      0.67      0.79        39

    accuracy                           0.81        74
   macro avg       0.84      0.82      0.81        74
weighted avg       0.85      0.81      0.81        74

Confusion Matrix:
[[34  1]
 [13 26]]


# **Cosine Similarity**

In [39]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

text_data = data["source_text"] + " " + data["plagiarized_text"]

# Split the combined text data
X_train_text, X_test_text, y_train, y_test = train_test_split(
    text_data, data["label"], test_size=0.2, random_state=42
)

# Now apply TF-IDF to the raw text data
vectorizer = TfidfVectorizer(ngram_range=(1, 3), stop_words='english', max_features=15000)
X_train_tfidf = vectorizer.fit_transform(X_train_text)
X_test_tfidf = vectorizer.transform(X_test_text)

# Compute cosine similarity between test and training texts
similarity_matrix = cosine_similarity(X_test_tfidf, X_train_tfidf)

# Define threshold for plagiarism detection
threshold = 0.44  # Adjust this based on dataset performance

# Predict plagiarism: If max similarity > threshold, label as 1 (plagiarized), else 0 (original)
y_pred = (similarity_matrix.max(axis=1) > threshold).astype(int)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

# Generate classification report and confusion matrix
classification_rep = classification_report(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

# Print results
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_rep)
print("Confusion Matrix:")
print(cm)

Accuracy: 0.6486486486486487
Classification Report:
              precision    recall  f1-score   support

           0       0.60      0.77      0.68        35
           1       0.72      0.54      0.62        39

    accuracy                           0.65        74
   macro avg       0.66      0.65      0.65        74
weighted avg       0.67      0.65      0.64        74

Confusion Matrix:
[[27  8]
 [18 21]]


# **SVM**

In [46]:
from sklearn.svm import SVC

# Instantiate the model
model = SVC(kernel='linear', random_state=42)
# Fit the model
model.fit(X_train, y_train)
# Make predictions
y_pred = model.predict(X_test)
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
# Generate classification report
classification_rep = classification_report(y_test, y_pred)
# Generate confusion matrix
cm = confusion_matrix(y_test, y_pred)
# Print results
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_rep)
print("Confusion Matrix:")
print(cm)

Accuracy: 0.8783783783783784
Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.89      0.87        35
           1       0.89      0.87      0.88        39

    accuracy                           0.88        74
   macro avg       0.88      0.88      0.88        74
weighted avg       0.88      0.88      0.88        74

Confusion Matrix:
[[31  4]
 [ 5 34]]


In [47]:
import pickle

pickle.dump(model,open("model.pkl",'wb'))
pickle.dump(tfidf_vectorizer, open('tfidf_vectorizer.pkl','wb'))

In [48]:
model = pickle.load(open('model.pkl','rb'))
tfidf_vectorizer = pickle.load(open('tfidf_vectorizer.pkl','rb'))

In [49]:
def detect(input_text):
    vectorized_text = tfidf_vectorizer.transform([input_text])
    result = model.predict(vectorized_text)
    return "Plagiarism Detected" if result[0] == 1 else "No Plagiarism"

In [50]:
# example ( it is a plagarized text)
input_text = 'Researchers have discovered a new species of butterfly in the Amazon rainforest.'
detect(input_text)

'Plagiarism Detected'

In [51]:
# example ( it has no plagiarism)
input_text = 'Playing musical instruments enhances creativity.'
detect(input_text)

'No Plagiarism'

In [52]:
# example ( it has no plagarism)
input_text = 'Practicing yoga enhances physical flexibility.'
detect(input_text)

'No Plagiarism'